# Numérisation

Pour présenter en entrée d'une fonction une image (ou un texte), il faut pouvoir le représenter sous forme numérique. \
Cette série de cinq exercices permet de préciser cette forme.

Mais avant, nous avons besoin :
- de charger le répertoire git du TD sur colab
- d'importer les fonctions qui nous serons utiles

In [ ]:
# package pour le TD
! git clone https://github.com/nanopiero/intro-ia-secondes.git

In [ ]:
# package pour le traitement du langage
! pip install datasets evaluate transformers[sentencepiece]

In [3]:
import sys
sys.path.append('intro-ia-secondes/utiles')
from utile import *
from transformers import pipeline

## **Exercice n°1** Manipulation de tables de nombres

In [ ]:
# Commande table : définition d'une ligne de nombres
a = table([1,2,3,4])
print(a)

In [ ]:
# Commande table : définition d'une table de nombres
b = table([[1,2],[3,4]])
print(b)

In [ ]:
# sommes et produit de deux tables de nombres:
print(b + b)
print(1 + b)
print(4 - b)
print(b*b)


In [ ]:
# Commande "repete" :
c = repete(a, 3) # 3 fois
print(c)

In [ ]:
# Commande "transpose" :
d = transpose(c) # échange les lignes et les colonnes
print(d)

In [ ]:
# commande suite_arithmetique:
n = 10
e = suite_arithmetique(n)

print(e)

**Question n°1** : A partir des exemples ci-dessus, construire la table d'addition et la table de multiplication des nombres de 0 à 20.

## **Exercice n°2** : une image en noir et blanc

Dans cet exercice, on retrouve le jeu d'images de chiffres manuscrits (MNIST) sur lequel
a travaillé Yann Lecun, l'un des "pères fondateurs" du Deep Learning.

In [ ]:
# commande preleve_image_MNIST : prélève la n ième image de MNIST
# et la cible correspondante (label)

image, label = preleve_image_MNIST(46) # c'est une image de taille 28 x 28

print(label)
afficher(image)

**Question n°1** : Avec une boucle *for*, appliquer *preleve_image_MNIST* pour afficher les 100 premières cibles. Afficher l'image d'un sept.

**Question n°2** : Pouvez-vous afficher un 7 noir sur fond blanc ?

**Question n°3** : Pouvez-vous construire une image du symbole $\infty$ ? Et une image du nombre 88 ?

## **Exercice n°3** : une image couleur

In [ ]:
# Commande ouvrir et commande afficher :
chemin_image = 'intro-ia-secondes/images/Carabus.jpg'
image = ouvrir(chemin_image)

afficher(image)

In [ ]:
# C'est une image RGB :

print(image[0]) # table des Rouges
# print(image[1]) # table des Verts
# print(image[2]) # table des Verts

In [ ]:
# Suppression des rouges :
image[0] = transposition(image[0])
afficher(image)

**Question n° 1** : Au lieu de transposer le canal des rouges, pouvez-vous supprimer le canal des rouges ?

**Question n° 2** : Pouvez-vous au contraire, n'afficher que le canal des rouges ?

**Question n°3** : Pouvez-vous transformer l'image RGB en une image en noir et blanc ?

## **Exercice n°4** : passer une image en entrée d'un "très gros réseau"

Maintenant que la représentation numérique d'une image sous forme de tables de nombres est bien comprise, voyons comment on applique un gros réseau de neurones dessus pour de la classification.

In [ ]:
# chargeons un modèle :
from torchvision import models, transforms
model = models.vgg16(pretrained=True)

In [ ]:
# Voyons sous le capot :
print(model)

**Question n°1** : A votre avis, quoi correspondent le premier chiffre (3) et le dernier chiffre visible (1000) ?

**Question n°2** : Le VGG est fourni par la bibliothèque pytorch. Pouvez-vous compter le nombre de poids du réseau facilement ?

In [7]:
# Get the total number of parameters in the model
total_params = sum(p.numel() for p in model.parameters())

# Print the total number of parameters
print(f"Total Parameters: {total_params}")

Total Parameters: 138357544


**Question n°3** : Reprenons la table "image" obtenue à partir de Carabus.jpg. Elle est au format numpy.array. Essayer de la donner en entrée du réseau et d'obtenir la classe la plus probable. \

**NB** : La liste des classes est disponible [ici](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a).

In [10]:
chemin_image = 'intro-ia-secondes/images/Carabus.jpg'
image = ouvrir(chemin_image)

**Question n°4** : Pouvez-vous donner la seconde classe la plus probable ?

**Question n°5** : Si l'on perd ou si l'on transpose un canal, le réseau s'en sort-il toujours ?

## **Exercice n°5**: Tokeniser un texte avant de le traduire.

Maintenant, voyons la façon dont sont numérisés les textes avant d'être donnés en entrée d'un réseau de neurone comme GPT ou autre (ici : le modèle [Helsinki-NLP](https://huggingface.co/Helsinki-NLP)). L'idée est de découper chaque phrase en éléments de bases qui sont chacun associé à un nombre entier. On parle de "tokénisation".

In [ ]:
# On charge le traducteur complet et on définit le tokeniseur, le modèle et un detokeniseur
# qui permet de repasser d'une suite de token à une phrase :
traducteur = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")

tokeniseur = lambda texte : traducteur.tokenizer(texte, return_tensors='pt')['input_ids']
detokeniseur = lambda tokens : traducteur.tokenizer.batch_decode(tokens, skip_special_tokens=True)[0]
reseau = lambda tokens : traducteur.model.eval().generate(tokens)

In [ ]:
a = "Question 1 : \n | Des entrées      différentes | \n | peuvent-elles conduire | \n | aux mêmes tokens ?| "
print(a)
print(tokeniseur(a))
print(detokeniseur(tokeniseur(a)))
print(tokeniseur(detokeniseur(tokeniseur(a))))

In [ ]:
# Le tokeniseur prend en entrée une phrase, et fournit en sortie une suite de nombres :

tokeniseur("Question 2 : quel est l'élément de texte associé au token n°99 ?")

In [ ]:
tokeniseur("Question 3 : un token peut-il correspondre à un mot entier ?")

Pour traduire un texte en anglais, il suffit de fournir la suite de tokens au modèle.
Par exemple :

In [ ]:
tokens = tokeniseur("Question 4 : pour un input donné, le modèle répond-il toujours la même chose ?")
new_tokens = reseau(tokens)
traduction = detokeniseur(new_tokens)

print(traduction)

**Question 5**: Pouvez-vous faire boguer ce traducteur ?

**Bilan**
* Les réseaux de neurones actuels peuvent prendre en entrée des images ou des textes représentés par des tables ou des suites de nombres.
* Par exemple, une image RGB est un empilement de trois tables : la première contient les intensités associées au rouge, la seconde, celles au vert et la troisième au bleu.
* Une tokénisation est une manière de nettoyer,puis de découper un texte en éléments de petite taille et enfin d'associer chaque élément à un entier. En général, ces éléments sont plus petits que des mots mais plus longs que de simples caractères.